In [ ]:
#匯入module
!pip install face_recognition

     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=71162d5327822552ce29ef842cb2ed50aa00d59f54a56c7c446e39e58e3259dd
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
#安裝 CUDA 在 Google Colab GPU's
!pip install mxnet-cu90==1.1.0

     |████████████████████████████████| 307.5 MB 48 kB/s 
     |████████████████████████████████| 5.0 MB 21.9 MB/s 
     |████████████████████████████████| 88 kB 6.2 MB/s 
     |████████████████████████████████| 132 kB 44.8 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
 

In [ ]:
#安裝 gspread 套件
!pip install --upgrade gspread

  Attempting uninstall: gspread
    Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [ ]:
#連結google sheet
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

#打開新工作表並添加一些數據
worksheet = gc.open('人臉簽到').sheet1

In [ ]:
#上傳圖片
from google.colab import files
import os

os.chdir('/content') 
path = 'data'
if not os.path.isdir(path):
    os.mkdir(path)

os.chdir('/content/data') 
uploaded = files.upload()

os.chdir('/content') 

Saving aaaaa.jpg to aaaaa.jpg
Saving bbbbb.jpg to bbbbb.jpg
Saving ccccc.jpg to ccccc.jpg
Saving ddddd.jpg to ddddd.jpg
Saving eeeee.jpg to eeeee.jpg
Saving fffff.jpg to fffff.jpg
Saving ggggg.jpg to ggggg.jpg
Saving jjjjjj.jpg to jjjjjj.jpg
Saving kkkkk.jpg to kkkkk.jpg
Saving lllll.jpg to lllll.jpg
Saving mmmmm.jpg to mmmmm.jpg
Saving ooooo.jpg to ooooo.jpg
Saving ppppp.jpg to ppppp.jpg


In [ ]:
# 將 JavaScript 對象轉換為 OpenCV 圖像的函數
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # 解碼 base64 圖像
  image_bytes = b64decode(js_reply.split(',')[1])
  # 將字節轉換為 numpy 數組
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # 將 numpy 數組解碼為 OpenCV BGR 圖像
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
# 使用網絡攝像頭作為輸入正確創建實時視頻流的 JavaScript
'''
在網絡攝像頭視頻上運行代碼比圖像複雜一些。
我們需要使用我們的網絡攝像頭作為輸入來啟動視頻流。
然後我們通過我們的程序（人臉檢測）運行每一幀並創建一個包含檢測邊界框的覆蓋圖像。
然後我們將邊界框圖像覆蓋回我們視頻流的下一幀。
'''
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;

    //刪除一幀
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    //通知瀏覽器我們想要產生動畫
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    //建立一幀
    async function createDom() {
      if (div !== null) {
        return stream;
      }
      
      //依指定的標籤名稱建立 HTML 元素
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      //依指定的標籤名稱建立 HTML 元素
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      //依指定的標籤名稱建立 HTML 元素    
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      //依指定的標籤名稱建立 HTML 元素
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      //依指定的標籤名稱建立 HTML 元素
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      //依指定的標籤名稱建立 HTML 元素
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
      /*=====================================================
      ***Document.createElement() 
      ***可以依指定的標籤名稱（tagName）建立 HTML 元素
      ***或是在未定義標籤名稱下建立一個 HTMLUnknownElement (en-US)
      ***將會建立指定的 XUL 元素
      ***在其它文件，則會建立一個 namespace URI 為 null 的元素。
      =====================================================*/
    }

    //架框
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
      
      //建立一个指向客户端中每一个盒子的边界矩形的矩形集合      
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
#import
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cmake
import dlib
from google.colab import files
import numpy as np
import cv2
import face_recognition
import time
from datetime import datetime,timezone,timedelta
import os
from IPython.display import Image

In [ ]:
#建立變數
path = 'data'
images = []
classNames = []
limit=False
cell_list = gc.open('人臉簽到').sheet1.range('B1:C2')

#圖片名
myList = os.listdir(path)
print(myList)

#姓名
for cl in myList:
  curImg = cv2.imread(f'{path}/{cl}')
  images.append(curImg)
  classNames.append(os.path.splitext(cl)[0])
print(classNames)

#使用CNN人臉檢測器返回圖像中人臉邊界框的二維數組
def findEncodings(images):
  encodeList = []
  for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeList.append(encode)
  return encodeList

#上傳至google sheet
def update():
  with open('Sign.csv','r') as f:
    myDataList = f.readlines()
    myDataListS=myDataList[len(myDataList)-1].split(',')
    worksheet.append_row([myDataListS[0],myDataListS[1]])

#標記
def markAttendance(name):
  notifylimit=False
  if not(os.path.isfile('Sign.csv')):
    with open('Sign.csv','w') as f:
      f.write('name,date,\n')
  with open('Sign.csv','r+') as f:
    myDataList = f.readlines()
    nameList = []
    for line in myDataList:
      entry = line.split(',')
      nameList.append(entry[0])
    if name not in nameList:
      notifylimit=True
      nowUTC=datetime.utcnow().replace(tzinfo=timezone.utc)
      nowUTC8 = nowUTC.astimezone(timezone(timedelta(hours=8))) # 轉換時區 -> 東八區
      dtString = nowUTC8.strftime("%Y-%m-%d %H:%M:%S")
      f.writelines(f'{name},{dtString},\n')
  return notifylimit
 
encodeListKnown = findEncodings(images)

print('====================')#分隔線

# 開始從網絡攝像頭傳輸視頻
video_stream()
# label for video
label_html = 'Talent Science Lab...'
# 將邊界框初始化為空
bbox = ''
count = 0 
while True: 
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
      break
  # 將 JS 響應轉換為 OpenCV 圖像
  img = js_to_image(js_reply["img"])
  # 為邊界框創建透明覆蓋
  bbox_array = np.zeros([480,640,4], dtype=np.uint8)
  imgS = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
  facesCurFrame = face_recognition.face_locations(imgS)
  encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
  
  for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
    matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
    faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
    matchIndex = np.argmin(faceDis)
    
    if faceDis[matchIndex]<0.50:
      # time.sleep(2)
      name = classNames[matchIndex].upper()
      limit = markAttendance(name)
    if limit:
      update()
      limit=False


  cv2.waitKey(1)

['mmmmm.jpg', 'lllll.jpg', 'GEM.jpg', 'aaaaa.jpg', 'kkkkk.jpg', 'ooooo.jpg', 'trump.jpg', 'yagoo.jpg', 'ggggg.jpg', 'ppppp.jpg', 'ddddd.jpg', 'jjjjjj.jpg', 'fffff.jpg', 'eeeee.jpg', 'biden.jpg', 'bbbbb.jpg', 'ccccc.jpg']
['mmmmm', 'lllll', 'GEM', 'aaaaa', 'kkkkk', 'ooooo', 'trump', 'yagoo', 'ggggg', 'ppppp', 'ddddd', 'jjjjjj', 'fffff', 'eeeee', 'biden', 'bbbbb', 'ccccc']
完成


<IPython.core.display.Javascript object>

#參考資料
How to install CUDA in Google Colab GPU's：https://stackoverflow.com/questions/50560395/how-to-install-cuda-in-google-colab-gpus \
人臉識別Dlib——68個特徵點：https://www.twblogs.net/a/5c37511ebd9eee35b21d596f \
Webcam Videos：https://colab.research.google.com/drive/1QnC7lV7oVFk5OZCm75fqbLAfD9qBy9bw?usp=sharing \
Face Attendance：https://www.computervision.zone/topic/attendance-project/